In [3]:
# !pip install gensim
# !pip install -U spacy
# !python -m spacy download en_core_web_trf
!pip install pyldavis

Processing c:\users\mgard\appdata\local\pip\cache\wheels\c9\21\f6\17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f\pyldavis-3.3.1-py2.py3-none-any.whl
Processing c:\users\mgard\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e\sklearn-0.0-py2.py3-none-any.whl


In [2]:
import spacy
spacy.load('en_core_web_trf')
from spacy.lang.en import English
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import pickle
import gensim
from gensim import corpora

C:\Users\mgard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mgard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mgard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mgard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [3]:
parser = English()
en_stop = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [4]:
file = open('Test_text.txt','r')
text_data = []
for line in file.readlines():
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)
# print(text_data)


In [5]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=1)
for topic in topics:
    print(topic)


I0406 14:07:42.604355  1112 dictionary.py:200] adding document #0 to Dictionary(0 unique tokens: [])
I0406 14:07:42.610338  1112 dictionary.py:205] built Dictionary(249 unique tokens: ['century', 'early', 'narrator', 'bamboozle', 'light']...) from 292 documents (total 710 corpus positions)
I0406 14:07:42.612334  1112 utils.py:448] Dictionary lifecycle event {'msg': "built Dictionary(249 unique tokens: ['century', 'early', 'narrator', 'bamboozle', 'light']...) from 292 documents (total 710 corpus positions)", 'datetime': '2021-04-06T14:07:42.611336', 'gensim': '4.0.1', 'python': '3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'created'}
I0406 14:07:42.619315  1112 utils.py:448] Dictionary lifecycle event {'fname_or_handle': 'dictionary.gensim', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-04-06T14:07:42.619315', 'gensim': '4.0.1', 'python': '3.7.3 (default, Mar 27 2019, 17:13

I0406 14:07:43.910888  1112 ldamodel.py:1185] topic #9 (0.100): 0.064*"pattern" + 0.046*"diffraction" + 0.036*"could" + 0.036*"second" + 0.036*"give" + 0.026*"words" + 0.019*"quantum" + 0.019*"formula" + 0.019*"describe" + 0.019*"constructive"
I0406 14:07:43.912883  1112 ldamodel.py:1185] topic #1 (0.100): 0.068*"electron" + 0.058*"louis" + 0.054*"would" + 0.045*"broglie" + 0.043*"associate" + 0.036*"particle" + 0.033*"property" + 0.033*"might" + 0.029*"light" + 0.023*"write"
I0406 14:07:43.916388  1112 ldamodel.py:1185] topic #4 (0.100): 0.081*"momentum" + 0.051*"light" + 0.046*"show" + 0.042*"divide" + 0.039*"right" + 0.034*"physicist" + 0.034*"always" + 0.023*"quantum" + 0.023*"physics" + 0.023*"young"
I0406 14:07:43.919379  1112 ldamodel.py:1185] topic #7 (0.100): 0.054*"things" + 0.050*"broglie" + 0.047*"relationship" + 0.041*"thought" + 0.037*"idea" + 0.033*"right" + 0.028*"square" + 0.028*"actual" + 0.028*"bright" + 0.026*"sometimes"
I0406 14:07:43.924365  1112 ldamodel.py:1063]

I0406 14:07:45.519916  1112 ldamodel.py:1185] topic #3 (0.100): 0.192*"light" + 0.109*"speed" + 0.087*"behavior" + 0.031*"exhibit" + 0.021*"broglie" + 0.021*"travel" + 0.021*"show" + 0.011*"would" + 0.011*"hole" + 0.011*"using"
I0406 14:07:45.521909  1112 ldamodel.py:1063] topic diff=0.029088, rho=0.301511
I0406 14:07:45.691584  1112 ldamodel.py:838] -6.126 per-word bound, 69.8 perplexity estimate based on a held-out corpus of 292 documents with 710 words
I0406 14:07:45.692583  1112 ldamodel.py:992] PROGRESS: pass 10, at document #292/292
I0406 14:07:45.770373  1112 ldamodel.py:1185] topic #3 (0.100): 0.197*"light" + 0.108*"speed" + 0.087*"behavior" + 0.030*"exhibit" + 0.023*"show" + 0.021*"broglie" + 0.021*"travel" + 0.011*"would" + 0.011*"hole" + 0.011*"using"
I0406 14:07:45.776360  1112 ldamodel.py:1185] topic #9 (0.100): 0.071*"pattern" + 0.053*"diffraction" + 0.036*"give" + 0.036*"second" + 0.036*"could" + 0.035*"words" + 0.019*"formula" + 0.019*"constructive" + 0.019*"destructive

I0406 14:07:46.970619  1112 utils.py:448] LdaState lifecycle event {'fname_or_handle': 'model5.gensim.state', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-04-06T14:07:46.970619', 'gensim': '4.0.1', 'python': '3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'saving'}
I0406 14:07:46.980592  1112 utils.py:618] saved model5.gensim.state
I0406 14:07:46.984583  1112 utils.py:448] LdaModel lifecycle event {'fname_or_handle': 'model5.gensim', 'separately': "['expElogbeta', 'sstats']", 'sep_limit': 10485760, 'ignore': ['id2word', 'state', 'dispatcher'], 'datetime': '2021-04-06T14:07:46.984583', 'gensim': '4.0.1', 'python': '3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'saving'}
I0406 14:07:46.986577  1112 utils.py:678] storing np array 'expElogbeta' to model5.gensim.expElogbeta.npy
I0406 14:07:46.996552  1112 ut

(0, '0.178*"wavelength"')
(1, '0.092*"electron"')
(2, '0.069*"double"')
(3, '0.213*"light"')
(4, '0.085*"momentum"')
(5, '0.064*"constant"')
(6, '0.090*"light"')
(7, '0.068*"right"')
(8, '0.104*"people"')
(9, '0.072*"pattern"')
